In [5]:
import selenium
from selenium import webdriver
import requests
import pandas as pd
from bs4 import BeautifulSoup

#import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH =r'C:\Users\hp\Downloads\chromedriver_win32\chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
#wd.get('https://google.com')

C:\Users\hp\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
<ipython-input-5-a28af588ac32>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [6]:
wd.get('https://google.com')

In [58]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('underwater sea sponge')

<ipython-input-58-adecc920dbad>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = wd.find_element_by_css_selector('input.gLFyf')


In [7]:
wd.quit()

In [53]:

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.co.in/search?q=amphimedon+compressa&tbm=isch&ved=2ahUKEwjiwNuRmI72AhVT3HMBHQOZAvwQ2-cCegQIABAA&oq=amphimedon+compressa&gs_lcp=CgNpbWcQAzIHCCMQ7wMQJzoFCAAQgAQ6CggjEO8DEOoCECdQwxBY8xtg4SFoAXAAeACAAc4HiAG3CZIBBzItMS42LTGYAQCgAQGqAQtnd3Mtd2l6LWltZ7ABCsABAQ&sclient=img&ei=bycSYqLNBdO4z7sPg7KK4A8&authuser=0&bih=625&biw=1349&hl=en#imgrc=-PsDlTwNgg9EbM"
    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [10]:
import PIL
from PIL import Image
import os
import time
import io
import hashlib

def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [54]:

def search_and_download(search_term:str,driver_path:str,target_path=r'C:\Users\hp\Downloads\SeaSponge\images', number_images=150):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [55]:
search_term= "amphimedon compressa"
search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

<ipython-input-54-2c13792827b9>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
<ipython-input-53-b4d89866cef7>:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 112 search results. Extracting links from 0:112


<ipython-input-53-b4d89866cef7>:32: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 212 search results. Extracting links from 112:212
Found: 150 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYxTyHptUYsxf76RR0WTq2MHpkPGBjRGYjQg&usqp=CAU - as C:\Users\hp\Downloads\SeaSponge\images\amphimedon_compressa\fe546c34da.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRERVi7JAn3zlq1Pb-hXK_-j5RsoX7eZkjZ7w&usqp=CAU - as C:\Users\hp\Downloads\SeaSponge\images\amphimedon_compressa\a6e736a685.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQCDcdvrbxNmyFC4ZplH2KmbTcEOG_YOJvZlg&usqp=CAU - as C:\Users\hp\Downloads\SeaSponge\images\amphimedon_compressa\601d25df2e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQhbPXByufT3PI0p-ymhANP3Rl1FuSfJuauYw&usqp=CAU - as C:\Users\hp\Downloads\SeaSponge\images\amphimedon_compressa\2607049e28.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGmYIQfpt8CrzIhINsjWM-xEgfY1kXsz7PyA&usqp=CAU - as C:\Users

In [65]:
from bs4 import *
import requests
import os
 
# CREATE FOLDER
def folder_create(images):
    try:
        folder_name = input("Enter Folder Name:- ")
        # folder creation
        os.mkdir(folder_name)
 
    # if folder exists with that name, ask another name
    except:
        print("Folder Exist with that name!")
        folder_create()
 
    # image downloading start
    download_images(images, folder_name)
 
 
# DOWNLOAD ALL IMAGES FROM THAT URL
def download_images(images, folder_name):
   
    # initial count is zero
    count = 0
 
    # print total images found in URL
    print(f"Total {len(images)} Image Found!")
 
    # checking if images is not zero
    if len(images) != 0:
        for i, image in enumerate(images):
            # From image tag ,Fetch image Source URL
 
                        # 1.data-srcset
                        # 2.data-src
                        # 3.data-fallback-src
                        # 4.src
 
            # Here we will use exception handling
 
            # first we will search for "data-srcset" in img tag
            try:
                # In image tag ,searching for "data-srcset"
                image_link = image["data-srcset"]
                 
            # then we will search for "data-src" in img
            # tag and so on..
            except:
                try:
                    # In image tag ,searching for "data-src"
                    image_link = image["data-src"]
                except:
                    try:
                        # In image tag ,searching for "data-fallback-src"
                        image_link = image["data-fallback-src"]
                    except:
                        try:
                            # In image tag ,searching for "src"
                            image_link = image["src"]
 
                        # if no Source URL found
                        except:
                            pass
 
            # After getting Image Source URL
            # We will try to get the content of image
            try:
                r = requests.get(image_link).content
                try:
 
                    # possibility of decode
                    r = str(r, 'utf-8')
 
                except UnicodeDecodeError:
 
                    # After checking above condition, Image Download start
                    with open(f"{folder_name}/images{i+1}.jpg", "wb+") as f:
                        f.write(r)
 
                    # counting number of image downloaded
                    count += 1
            except:
                pass
 
        # There might be possible, that all
        # images not download
        # if all images download
        if count == len(images):
            print("All Images Downloaded!")
             
        # if all images not download
        else:
            print(f"Total {count} Images Downloaded Out of {len(images)}")
 
# MAIN FUNCTION START
def main(url):
   
    # content of URL
    r = requests.get(url)
 
    # Parse HTML Code
    soup = BeautifulSoup(r.text, 'html.parser')
 
    # find all images in URL
    images = soup.findAll('img')
 
    # Call folder create function
    folder_create(images)
 
 
# take url
url = input("Enter URL:- ")
 
# CALL MAIN FUNCTION
main(url)

Enter URL:- https://www.gettyimages.in/photos/sea-sponge?assettype=image&page=3&phrase=sea%20sponge&sort=mostpopular&license=rf%2Crm
Enter Folder Name:- sponges
Total 0 Image Found!


In [61]:
C:\Users\hp\Downloads\SeaSponge\images\amphimedon_compressa2\

SyntaxError: unexpected character after line continuation character (<ipython-input-61-5052f13dfd80>, line 1)

In [67]:
import re
import requests
from bs4 import BeautifulSoup

site = 'https://www.gettyimages.in/photos/sea-sponge?assettype=image&phrase=sea%20sponge&sort=mostpopular&license=rf,rm&page=2'

response = requests.get(site)

soup = BeautifulSoup(response.text, 'html.parser')
img_tags = soup.find_all('img')

urls = [img['src'] for img in img_tags]


for url in urls:
    filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
    if not filename:
         print("Regex didn't match with the url: {}".format(url))
         continue
    with open(filename.group(1), 'wb') as f:
        if 'http' not in url:
            # sometimes an image source can be relative 
            # if it is provide the base url which also happens 
            # to be the site variable atm. 
            url = '{}{}'.format(site, url)
        response = requests.get(url)
        f.write(response.content)